# Hands Classification
We have recording of hands in 3 different scenarios.  
1. Spontan- two hand moving freely.
2. Sync- two hand moving, trying to be synchronized.
3. Alone- one hand moving freely.  

Our target is to classify hands to one of the categories according to their recorded movements and position. 

In [1]:
#Libraries import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

In [2]:
#understanding the data
#example data:
Evyatar_spon=pd.read_csv('Datasets/Hands/Training/Evyatar Cohen/Evyatar636771052727603804Spontan.csv')

In [3]:
Evyatar_spon.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,335.9984,128556,right,2,128.63570,224.0294,-12.29588,-18.26243,-140.54450,-117.57790,...,-1.588251,177.5125,173.5658,-14.569630,276.8753,-33.68264,91.61740,0.0,1.250576,0.0
1,335.9984,128556,left,2,-93.24845,143.9423,-37.81597,-109.09820,240.38980,-67.62959,...,0.873458,-147.0455,109.9380,-5.326688,-329.8577,-34.41571,93.88171,0.0,0.014736,0.0
2,336.0150,128558,right,2,129.15550,221.9923,-15.00181,47.36464,-64.25022,-168.85850,...,-1.528183,177.7894,171.1952,-14.677850,276.0645,-33.15913,97.88171,0.0,1.069930,0.0
3,336.0150,128558,left,2,-94.86554,148.5542,-39.00158,-80.99012,278.60220,-66.79356,...,0.879703,-148.2022,113.0909,-7.375025,-326.9279,-39.97873,85.98000,0.0,0.000000,0.0
4,336.0315,128560,right,2,129.87070,220.7944,-18.38681,35.05422,-84.74242,-205.72030,...,-1.461710,178.3730,169.9159,-15.246240,276.1767,-30.72506,104.13920,0.0,0.916969,0.0


In [4]:
Evyatar_spon.loc[0].index

Index(['Time', ' Frame ID', ' Hand Type', ' # hands', ' Position X',
       ' Position Y', ' Position Z', ' Velocity X', ' Velocity Y',
       ' Velocity Z', ' Pitch', ' Roll', ' Yaw', ' Wrist Pos X',
       ' Wrist Pos Y', ' Wrist Pos Z', ' Elbow pos X', ' Elbow Pos Y',
       ' Elbow Pos Z', ' Grab Strenth', ' Grab Angle', ' Pinch Strength'],
      dtype='object')

Every ~1/6 second we have a "picture" of two hand- left and right.  
Each "picture" contains information about the hands placement, velocity and position.  
We will treat each pair of hands in a frame as one instance (single object).  
The "Alone" files will be paired with the file "HandRight.csv".  

## Data Importing

In [19]:
#data import
from os import listdir
folder=('Datasets/Hands/Training/')
Spon, Sync, Alone=list(), list(), list()
for participant in listdir(folder):
    for file in listdir(folder+participant+'/'):
        if file.endswith('Spontan.csv'):
            spon=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Sync.csv'):
            sync=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Alone.csv'):
            alone=pd.read_csv(folder+participant+'/'+file)
    Spon.append(spon)
    Sync.append(sync)
    Alone.append(alone)
print('Example df length: '+str(len(Alone[1])))#frames in Alone second dataframe
print ('no. of df in Sync '+str(len(Sync)))

Example df length: 4575
no. of df in Sync 9


We now have a list for every scenario.

We will remove the first 7 seconds from each recording (time to get synchronized).

In [21]:
for scenario in (Spon,Sync,Alone):
    for df in scenario:
        df.drop(range(2*6*7),inplace=True)
len(Alone[1])

4491

We'll do the same for the test data

In [24]:
#import test data
folder=('Datasets/Hands/Validation/')
Spon_test, Sync_test, Alone_test=list(), list(), list()
for participant in listdir(folder):
    for file in listdir(folder+participant+'/'):
        if file.endswith('Spontan.csv'):
            spon=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Sync.csv'):
            sync=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Alone.csv'):
            alone=pd.read_csv(folder+participant+'/'+file)
    Spon_test.append(spon)
    Sync_test.append(sync)
    Alone_test.append(alone)
print('length before reduce: '+str(len(Spon_test[0])))
#remove first 7 seconds
for scenario in (Spon_test,Sync_test,Alone_test):
    for df in scenario:
        df.drop(range(2*6*7),inplace=True)
print('length before reduce: '+str(len(Spon_test[0])))

length before reduce: 8969
length before reduce: 8885
